# Генеративні мережі

Рекурентні нейронні мережі (RNN) та їхні варіанти з керованими осередками, такі як осередки довготривалої короткочасної пам'яті (LSTM) і керовані рекурентні блоки (GRU), забезпечили механізм для моделювання мови, тобто вони можуть навчитися впорядковувати слова та передбачати наступне слово в послідовності. Це дозволяє використовувати RNN для **генеративних завдань**, таких як звичайне генерування тексту, машинний переклад і навіть створення підписів до зображень.

У архітектурі RNN, яку ми обговорювали в попередньому розділі, кожен блок RNN генерував наступний прихований стан як вихід. Однак ми також можемо додати ще один вихід до кожного рекурентного блоку, що дозволить нам генерувати **послідовність** (яка дорівнює за довжиною оригінальній послідовності). Більше того, ми можемо використовувати блоки RNN, які не приймають вхідні дані на кожному кроці, а лише отримують початковий вектор стану, після чого генерують послідовність виходів.

У цьому ноутбуці ми зосередимося на простих генеративних моделях, які допомагають нам генерувати текст. Для простоти побудуємо **мережу на рівні символів**, яка генерує текст літера за літерою. Під час навчання нам потрібно взяти текстовий корпус і розбити його на послідовності літер.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Створення словника символів

Щоб побудувати генеративну мережу на рівні символів, потрібно розбити текст на окремі символи, а не на слова. Шар `TextVectorization`, який ми використовували раніше, цього зробити не може, тому у нас є два варіанти:

* Завантажити текст вручну і виконати токенізацію "вручну", як у [цьому офіційному прикладі Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Використати клас `Tokenizer` для токенізації на рівні символів.

Ми оберемо другий варіант. `Tokenizer` також можна використовувати для токенізації на рівні слів, тому можна легко переключитися з токенізації символів на токенізацію слів.

Для токенізації на рівні символів потрібно передати параметр `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Ми також хочемо використовувати один спеціальний токен для позначення **кінця послідовності**, який ми називатимемо `<eos>`. Додамо його вручну до словника:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Навчання генеративної RNN для створення заголовків

Ось як ми будемо навчати RNN створювати заголовки новин. На кожному кроці ми беремо один заголовок, який подається в RNN, і для кожного вхідного символу ми просимо мережу згенерувати наступний вихідний символ:

![Зображення, що демонструє приклад генерації слова 'HELLO' за допомогою RNN.](../../../../../translated_images/uk/rnn-generate.56c54afb52f9781d.webp)

Для останнього символу нашої послідовності ми попросимо мережу згенерувати токен `<eos>`.

Головна відмінність генеративної RNN, яку ми використовуємо тут, полягає в тому, що ми будемо брати вихідні дані з кожного кроку RNN, а не лише з фінальної комірки. Це можна реалізувати, вказавши параметр `return_sequences` для комірки RNN.

Таким чином, під час навчання вхідними даними для мережі буде послідовність закодованих символів певної довжини, а вихідними — послідовність тієї ж довжини, але зміщена на один елемент і завершена токеном `<eos>`. Мініпакет складатиметься з кількох таких послідовностей, і нам потрібно буде використовувати **доповнення** (padding), щоб вирівняти всі послідовності.

Давайте створимо функції, які трансформуватимуть для нас набір даних. Оскільки ми хочемо доповнювати послідовності на рівні мініпакетів, спочатку ми згрупуємо набір даних, викликавши `.batch()`, а потім застосуємо `map`, щоб виконати трансформацію. Таким чином, функція трансформації прийматиме цілий мініпакет як параметр:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Кілька важливих речей, які ми робимо тут:
* Спочатку ми витягуємо фактичний текст із тензора рядків
* `text_to_sequences` перетворює список рядків у список тензорів цілих чисел
* `pad_sequences` доповнює ці тензори до їх максимальної довжини
* Нарешті, ми виконуємо one-hot кодування всіх символів, а також робимо зсув і додаємо `<eos>`. Незабаром ми побачимо, чому нам потрібні символи у форматі one-hot.

Однак ця функція є **Pythonic**, тобто її не можна автоматично перекласти в обчислювальний граф Tensorflow. Ми отримаємо помилки, якщо спробуємо використовувати цю функцію безпосередньо у функції `Dataset.map`. Нам потрібно обгорнути цей виклик Pythonic, використовуючи обгортку `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Примітка**: Розрізнення між функціями перетворення Pythonic і Tensorflow може здатися занадто складним, і ви можете запитати, чому ми не трансформуємо набір даних за допомогою стандартних функцій Python перед передачею його в `fit`. Хоча це, безумовно, можливо, використання `Dataset.map` має величезну перевагу, оскільки конвеєр перетворення даних виконується за допомогою обчислювального графа Tensorflow, який використовує переваги обчислень на GPU і мінімізує необхідність передачі даних між CPU/GPU.

Тепер ми можемо побудувати нашу генеративну мережу і розпочати навчання. Вона може базуватися на будь-якій рекурентній комірці, яку ми обговорювали в попередньому розділі (простій, LSTM або GRU). У нашому прикладі ми використаємо LSTM.

Оскільки мережа приймає символи як вхідні дані, а розмір словника досить невеликий, нам не потрібен шар вбудовування, однокодовий вхід може безпосередньо передаватися в комірку LSTM. Вихідний шар буде класифікатором `Dense`, який перетворює вихід LSTM у номери токенів у форматі one-hot.

Крім того, оскільки ми працюємо зі змінною довжиною послідовностей, ми можемо використовувати шар `Masking`, щоб створити маску, яка ігноруватиме заповнені частини рядка. Це не є строго необхідним, оскільки нас не дуже цікавить усе, що виходить за межі токена `<eos>`, але ми використаємо його для того, щоб отримати певний досвід роботи з цим типом шару. `input_shape` буде `(None, vocab_size)`, де `None` вказує на послідовність змінної довжини, а вихідна форма також буде `(None, vocab_size)`, як ви можете побачити з `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Генерація результату

Тепер, коли ми навчили модель, ми хочемо використати її для генерації результатів. Перш за все, нам потрібен спосіб декодувати текст, представлений у вигляді послідовності чисел-токенів. Для цього ми могли б скористатися функцією `tokenizer.sequences_to_texts`; однак, вона не дуже добре працює з токенізацією на рівні символів. Тому ми візьмемо словник токенів із токенайзера (який називається `word_index`), створимо зворотну мапу і напишемо власну функцію декодування:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Тепер давайте почнемо генерацію. Ми почнемо з деякого рядка `start`, закодуємо його в послідовність `inp`, а потім на кожному кроці будемо викликати нашу мережу, щоб визначити наступний символ.

Вихід мережі `out` — це вектор із `vocab_size` елементів, що представляють ймовірності кожного токена, і ми можемо знайти номер найбільш ймовірного токена, використовуючи `argmax`. Потім ми додаємо цей символ до згенерованого списку токенів і продовжуємо генерацію. Цей процес генерації одного символу повторюється `size` разів, щоб згенерувати необхідну кількість символів, і ми завершуємо раніше, якщо зустрічаємо `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Вибірка результатів під час навчання

Оскільки у нас немає корисних метрик, таких як *точність*, єдиний спосіб побачити, що наша модель стає кращою, — це **вибірка** згенерованих рядків під час навчання. Для цього ми будемо використовувати **зворотні виклики** (callbacks), тобто функції, які ми можемо передати до функції `fit`, і які будуть викликатися періодично під час навчання.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Цей приклад вже генерує досить хороший текст, але його можна покращити кількома способами:

* **Більше тексту**. Ми використали лише заголовки для нашого завдання, але ви можете спробувати працювати з повним текстом. Пам’ятайте, що RNN не дуже добре справляються з довгими послідовностями, тому має сенс або розбивати їх на коротші речення, або завжди тренувати на фіксованій довжині послідовності з певним попередньо визначеним значенням `num_chars` (наприклад, 256). Ви можете спробувати змінити наведений вище приклад на таку архітектуру, використовуючи [офіційний підручник Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) як натхнення.

* **Багатошаровий LSTM**. Варто спробувати 2 або 3 шари LSTM-комірок. Як ми згадували в попередньому розділі, кожен шар LSTM витягує певні шаблони з тексту, і у випадку генератора на рівні символів можна очікувати, що нижчий рівень LSTM буде відповідати за витягування складів, а вищі рівні — за слова та їх комбінації. Це можна легко реалізувати, передавши параметр кількості шарів до конструктора LSTM.

* Ви також можете спробувати експериментувати з **GRU-комірками** і перевірити, які з них працюють краще, а також з **різними розмірами прихованих шарів**. Занадто великий прихований шар може призвести до перенавчання (наприклад, мережа буде запам’ятовувати точний текст), а менший розмір може не дати хорошого результату.


## Генерація м'якого тексту та температура

У попередньому визначенні `generate` ми завжди обирали символ із найвищою ймовірністю як наступний символ у згенерованому тексті. Це призводило до того, що текст часто "зациклювався" на одних і тих самих послідовностях символів знову і знову, як у цьому прикладі:
```
today of the second the company and a second the company ...
```

Однак, якщо подивитися на розподіл ймовірностей для наступного символу, може виявитися, що різниця між кількома найвищими ймовірностями не є значною, наприклад, один символ може мати ймовірність 0.2, а інший — 0.19 тощо. Наприклад, при пошуку наступного символу в послідовності '*play*', наступним символом може однаково бути пробіл або **e** (як у слові *player*).

Це приводить нас до висновку, що не завжди "справедливо" обирати символ із вищою ймовірністю, оскільки вибір другого за величиною ймовірності також може привести до осмисленого тексту. Більш розумно **вибирати випадкові** символи з розподілу ймовірностей, який надає вихід мережі.

Це вибіркове обрання можна здійснити за допомогою функції `np.multinomial`, яка реалізує так званий **мультиноміальний розподіл**. Функція, яка реалізує цю **м'яку** генерацію тексту, визначена нижче:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Ми ввели ще один параметр, який називається **temperature** (температура), і він використовується для визначення того, наскільки сильно ми повинні дотримуватися найвищої ймовірності. Якщо температура дорівнює 1.0, ми виконуємо справедливе мультиноміальне вибіркове моделювання, а коли температура наближається до нескінченності - всі ймовірності стають рівними, і ми випадково вибираємо наступний символ. У наведеному нижче прикладі ми можемо спостерігати, що текст стає безглуздим, коли ми надто сильно збільшуємо температуру, і він нагадує "циклічний" текст, створений жорстким алгоритмом, коли температура наближається до 0.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
